In [10]:
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-10-29 15:27:36.850694: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-10-29 15:27:36.850789: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-29 15:27:41.907516: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-10-29 15:27:41.907554: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-29 15:27:41.913394: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: Fuzel
2022-10-29 15:27:41.913626: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: Fuzel


In [8]:
!pip install negspacy

  Created wheel for negspacy: filename=negspacy-1.0.3-py3-none-any.whl size=10434 sha256=3ffa193a3fb3b5fd88f4b5007242c591ac3a4296643622d27bb9374fb7b0915e
  Stored in directory: c:\users\fuzel\appdata\local\pip\cache\wheels\ec\4b\5b\ae823d4b232c1342724ec93edab9f3dca6403679d93e0daa25
Successfully built negspacy


In [6]:
!pip install spacy

  Attempting uninstall: smart-open
    Found existing installation: smart-open 5.1.0
    Uninstalling smart-open-5.1.0:
      Successfully uninstalled smart-open-5.1.0


In [60]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import collections
import re
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import accuracy_score, classification_report
import spacy
from negspacy.negation import Negex

In [3]:
stop_words = set(stopwords.words('english'))

In [26]:
l = ["haven't","hadn't","not","don't","aren't","less","none","couldn't","doesn't","hasn't","isn't","mightn't","mustn't","needn't","shan't","shouldn't","wasn't","weren't","won't","wouldn't",""]
for w in l:
    try:
        stop_words.remove(w)
    except:
        pass

In [13]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("negex", config={"ent_types":["PERSON","ORG"]})

In [54]:
#df1 = pd.read_csv('Corona_Bodies.csv',encoding='latin-1')
#df2 = pd.read_csv('Corona_stance.csv',encoding='latin-1')
#combined_df = df2
#combined_df['articleBody'] = df1['articleBody']
#combined_df

df1 = pd.read_csv('Economic_Bodies.csv',encoding='latin-1')
df2 = pd.read_csv('Economic_Stance.csv',encoding='latin-1')
combined_df = df2
combined_df['articleBody'] = df1['articleBody']
combined_df

,Headline,Stance,Body ID,articleBody
0,Elon Musk says a global recession could last u...,agree,1,Tesla founder and CEO Elon Musk thinks the glo...
1,Heres how venture capital is helping to lift ...,discuss,2,Even though Latinos are the second-largest eth...
2,"U.S. GDP accelerated at 2.6% pace in Q3, bette...",agree,3,The U.S. economy posted its first period of po...
3,"Tatas, Lockheed Martin in talks to make Sikors...",discuss,4,The Tatas are in talks with Lockheed Martin fo...
4,U.S. GDP bounces back but economic outlook rem...,agree,5,The Commerce Department analysis found U.S. gr...
...,...,...,...,...
195,Britains Broken Capitalism Model and the Poli...,discuss,196,New Prime Minister Liz Truss has dominated hea...
196,How Private Investors Mortgaged UKs Utilities...,discuss,197,"Theres no Magic Money Tree, Theresa May famou..."
197,Economic Sobering Predictions,discuss,198,This article was written by the Economic Times...
198,Survey: Half Of Social Workers Thinking Of Qui...,discuss,199,Nearly half (49 per cent) of social workers sa...


In [55]:
a = []
b = []
c = []
count = 0
ps = PorterStemmer()
for _id in range(0,combined_df.shape[0]):
    head_line = combined_df.iloc[_id]['Headline']
    art_body = combined_df.iloc[_id]['articleBody']
    head_line = re.sub(r"\n|\'|-|,|\)|\(|\:|\?|@|‘|’|\*|\"|\$",'',head_line)
    head_line = ' '.join([i for i in head_line.split(' ') if i not in stop_words])
    head_line = ' '.join([ps.stem(i) for i in head_line.split(' ')])
    art_body = re.sub(r"\n|\'|-|,|\)|\(|\:|\?|@|‘|’|\*|\"|\$",'',art_body)
    art_body = ' '.join([i.lower() for i in art_body.split(' ') if i not in stop_words])
    art_body = ' '.join([ps.stem(i) for i in art_body.split(' ')])
    article_counter = collections.Counter(art_body)
    neg_body = []
    doc1 = nlp(art_body)
    for e in doc1.ents:
        neg_body.append(tuple([e.text,str(e._.negex)]))
    neg_head_line = []
    doc = nlp(head_line)
    for e in doc.ents:
        neg_head_line.append(tuple([e.text,str(e._.negex)]))
    count = 0
    head_line_counter = collections.Counter(head_line)
    try:
        for k in head_line_counter.keys():
            count = count+article_counter[k]
    except:
        count = count + 0
    if count<=1:
        a.append(head_line)
        b.append(art_body)
        c.append('unrelated')
    else:
        if count>=len(head_line) and count>=5:
            a.append(head_line)
            b.append(art_body)
            c.append('agree')
        elif count<len(head_line) and count >= 2:
            a.append(head_line)
            b.append(art_body)
            c.append('discuss')
        else:
            for index, tup in enumerate(neg_head_line):
                for i,j in enumerate(neg_body):
                    if index == i:
                        if tup != j:
                            a.append(head_line)
                            b.append(art_body)
                            c.append('disagree')

In [56]:
df = pd.DataFrame()
df['Headline'] = a
df['articleBody'] = b
df['Stance'] = c

In [57]:
df['Stance'].unique()

array(['agree', 'discuss'], dtype=object)

In [58]:
accuracy_score(combined_df['Stance'],df['Stance'])

0.25

In [61]:
print(classification_report(combined_df['Stance'],df['Stance']))

              precision    recall  f1-score   support

       agree       0.25      1.00      0.40        50
    disagree       0.00      0.00      0.00        50
     discuss       0.00      0.00      0.00        50
   unrelated       0.00      0.00      0.00        50

    accuracy                           0.25       200
   macro avg       0.06      0.25      0.10       200
weighted avg       0.06      0.25      0.10       200



C:\Users\fuzel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fuzel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fuzel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


End of the code